In [1]:
import os
import time
import copy
import json
from datetime import datetime
from elasticsearch import Elasticsearch

In [2]:
def timestamp_2_utc(src):
    return 'T'.join(str(datetime.utcfromtimestamp(int(src))).split(' '))

print(timestamp_2_utc(1528948005))

def local_2_utctimestamp(*src):
    if len(src) == 0:
        local = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    else:
        local = src[0]
    return int(str(time.mktime(datetime.strptime(local, '%Y-%m-%d %H:%M:%S').timetuple()))[:-2])

print(local_2_utctimestamp())
print(local_2_utctimestamp("2018-06-13 20:46:45"))
#print(local_2_utctimestamp("2018-06-28T05:01:41.656Z"))

2018-06-14T03:46:45
1530217596
1528948005


In [3]:
ELS_TDS_INDEX = 'helios_data*'

SELECT_QUERY_BODY = {
    'query': {
        'bool': {
            'must': [
                {"exists": {'field': 'sqlbatch'}},
                {"range": {
                    "@timestamp": {
                        "gte": 0,
                        "lte": 0
                    }
                }
                }
            ]
        }
    }
}

'''
SELECT_OTHER_BODY = {
    'query': {
        'bool': {
            'must': [
                {"range": {
                    "@timestamp": {
                        "gte": 0,
                        "lte": 0
                    }
                }
                }
            ]
        }
    }
}
'''

SELECT_OTHER_BODY = {
  "query": {
    "range": {
      "@timestamp": {
        "gte": 0,
        "lte": 0
      }
    }
  }
  #"sort": [
  #  {
  #    "ts": {
  #      "order": "desc"
  #    }
  #  }
  #]
}
#print(type(SELECT_OTHER_BODY['query']))
#print(type(SELECT_OTHER_BODY['query']['range']))
#print(type(SELECT_OTHER_BODY['query']['range']['@timestamp']["lte"]))

In [10]:
def connect_check_wrapper(func):
    def connect_handler(self, *args, **kwargs):
        if self._es is None:
            try:
                self.connect()
            except Exception as e:
                EXCEPT(e)
        return func(self, *args, **kwargs)
    return connect_handler

class ElasticSearch(object):

    def __init__(self, host, port, user, password):
        self._els_host = host
        self._els_port = port
        self._user = user
        self._password = password
        self._es = None
        #INIT("ElasticSearch class inited")

    def connect(self):
        elastic = ('http://%s:%s@%s:%s' %(self._user, self._password, self._els_host, self._els_port))
        self._es = Elasticsearch([elastic])

    @connect_check_wrapper
    def get_sql_select(self, time_start, time_end):
        query_body = copy.copy(SELECT_QUERY_BODY)
        for entry in query_body['query']['bool']['must']:
            if 'range' in entry.keys() and "@timestamp" in entry['range'].keys():
                entry['range']["@timestamp"]["gte"] = timestamp_2_utc(time_start)
                entry['range']['@timestamp']["lte"] = timestamp_2_utc(time_end)
        print("query %s, start %s, to %s" % (query_body, time_start, time_end))
        return self._es.search(index=ELS_TDS_INDEX, body=query_body)
    
    @connect_check_wrapper
    def get_others_select(self, time_start, time_end, sqlbatch_dict, cut=10):
        t0_pcap = sqlbatch_dict.get('ts')
        #print(t0_pcap)
        query_body = copy.copy(SELECT_OTHER_BODY)
        query_body['query']['range']["@timestamp"]["gte"] = timestamp_2_utc(time_start)
        query_body['query']['range']['@timestamp']["lte"] = timestamp_2_utc(time_end)
        print("query %s, start %s, to %s" % (query_body, time_start, time_end))        
        res = self._es.search(index=ELS_TDS_INDEX, body=query_body)
        
        hits = result.get('hits')
        all_hits = hits.get('hits')
        for i in all_hits:
            print(i.get('_source'))
            #print(i.get('_source').get('ts'))
            diff = i.get('_source').get('ts')-t0_pcap
            #if diff < cut and diff > 0:
            #    print(i.get('_source'))
        return None


start_els_str="2018-06-06 00:00:00"
end_els_str="2018-06-29 00:00:00"
print(local_2_utctimestamp(start_els_str))
print(local_2_utctimestamp(end_els_str))
start_els_str="2018-06-27 00:00:00"
end_els_str="2018-06-29 00:00:00"
print(local_2_utctimestamp(start_els_str))
print(local_2_utctimestamp(end_els_str))

#els = ElasticSearch("192.168.7.17", 9200, "elastic", "helios12$")
els = ElasticSearch("192.168.7.8", 9200, "elastic", "helios12$")

#result = els.get_sql_select(local_2_utctimestamp(start_els_str), local_2_utctimestamp(end_els_str))
#hits = result.get('hits')
#all_hits = hits.get('hits')
#for i in all_hits:
#    source = i.get('_source')
#    print(source)
#    print(source.get('ts'))

test_sqlbatch_dict = {'id.resp_p': 1433, 'sqlbatch': '\x16\x12\x02e\x89u\x01select * from EXT_ORDER_NEW\r\nWhere UPDSTATUS IS NUL', 'id.orig_h': '192.168.3.61', '@version': '1', 'id.orig_p': 61617, 'id.resp_h': '172.16.1.58', 'uid': 'CVdhbP1Hqo0ay9iUfl', 'ts': 1521699510.225344, '@timestamp': '2018-06-14T03:46:44.999Z'}
els.get_others_select(local_2_utctimestamp(start_els_str), local_2_utctimestamp(end_els_str), test_sqlbatch_dict, 1)

1528268400
1530255600
1530082800
1530255600
query {'query': {'range': {'@timestamp': {'gte': '2018-06-27T07:00:00', 'lte': '2018-06-29T07:00:00'}}}}, start 1530082800, to 1530255600
{'ts': 1530145293.45517, 'id.resp_h': '192.168.8.13', '@version': '1', 'id.orig_h': '192.168.8.15', 'id.resp_p': 1433, 'uid': 'CehnAGR0nHwC7BCYh', 'sqlbatch': "select tab4col2, country, city from pii_4th where country = 'USA", '@timestamp': '2018-06-28T00:25:28.953Z', 'id.orig_p': 41360}
{'ts': 1530145291.225971, 'id.resp_h': '192.168.8.13', '@version': '1', 'id.orig_h': '192.168.8.15', 'id.resp_p': 1433, 'uid': 'CigJBD3oECk6ZLUet3', 'sqlbatch': "select tab2col1, firstname from pii_2nd where gender = 'female", '@timestamp': '2018-06-28T03:11:02.822Z', 'id.orig_p': 41360}
{'ts': 1528997199.928567, 'id.resp_h': '192.168.7.155', '@version': '1', 'id.orig_h': '192.168.7.141', 'id.resp_p': 1433, 'uid': 'CPQ2tj3ev9oPSCsrzb', 'sqlbatch': '\x16\x12\x02\x01select * from phon', '@timestamp': '2018-06-27T17:36:29.495Z

### End of this file